Быстрый старт
===================

Работа с данными
-----------------
В PyTorch существует два базовых примитива для работы с данными <https://pytorch.org/docs/stable/data.html>:
``torch.utils.data.DataLoader`` и ``torch.utils.data.Dataset``.
``Dataset`` хранит сэмплы и соответстующие им значения целевой переменной (corresponding labels), а ``DataLoader`` wraps an iterable around the ``Dataset``.


In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch предлагает специальные библиотеки:
1. [TorchText](https://pytorch.org/text/stable/index.html),
2. [TorchVision](https://pytorch.org/vision/stable/index.html), 
3. [TorchAudio](https://pytorch.org/audio/stable/index.html),

которые содержат в себе некоторые наборы данных. 
В данном туторе мы будем использовать TorchVision dataset.

Модуль ``torchvision.datasets`` содержит объекты класса ``Dataset`` многих наборов данных, таких как CIFAR, COCO (https://pytorch.org/vision/stable/datasets.html). 
В данном туторе мы будем использовать датасет `FashionMNIST`. В каждом подобном датасете `TorchVision` есть два аргумента: ``transform`` и ``target_transform``, чтобы менять объекты и соответствующие им целевые переменные (labels).

In [3]:
# Загружаем тренировочную выборку из библиотеки datasets.
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

# Загружаем тестовую выборку из библиотеки datasets. 
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

print(f"Класс объектов: {type(training_data)}, {type(test_data)}")

Класс объектов: <class 'torchvision.datasets.mnist.FashionMNIST'>, <class 'torchvision.datasets.mnist.FashionMNIST'>


Помним, что `FashinMNIST` наследуется от базового класса `Dataset`. Объекты класса `Dataset` являются аргументами для объектов класса `DataLoader`. Что это значит? Это означает что наш набор данных (выборка) будет обернут в итерируемый объект, что позволит сделать:
1. автоматическая пакетная обрабка (batching)
2. выборку (sampling)
3. переасовку (shuffling)
4. многопроцессорную загрузку данных (multiprocess data loading)

Ниже представлен пример, мы задаем размер батча 64, т.е. каждый элемент в итерации загрузчика данных будет возвращать батч из 64 объектов (наборов признаков) и соответствующих им ответов (labels)

In [4]:
batch_size = 64

# Создание DataLoader
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape} {X.dtype}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28]) torch.float32
Shape of y: torch.Size([64]) torch.int64


In [5]:
train_dataloader.dataset

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [6]:
train_dataloader.dataset.data.size()

torch.Size([60000, 28, 28])

In [7]:
test_dataloader.dataset

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [8]:
test_dataloader.dataset.data.size()

torch.Size([10000, 28, 28])

Узнать об этих классах можно в ноутбуке ***03_data_tutorial***.

--------------




Создание модели нейронной сети
------------------
Для определения нейронной сети в PyTorch нужно создать класс, который наследуется от `nn.Module`. Внутри класса нужно обязательно определеить базовые функции ``__init__`` и ``forward``. 
1. ``__init__`` - здесь мы определяем сколько и какие слои будут в нашей сетке.
2. ``forward`` - здесь указывается в каком порядке будут проходить данные через сетку.

Для ускорения работы обучения нейронной сети мы будет переносить ее на GPU, если он доступен.

In [9]:
# Определить CPU или GPU 
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using '{device}' device")

Using 'cuda' device


In [10]:
# Создаем модель
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), 
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [11]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Узнать больше о построении моделей нейронных сетей в PyTorch можно в ноутбуке ***05_buildmodel_tutorial***.


--------------




Оптимизация параметров модели
----------------------------------------

Чтобы натренеровать модель (подобрать оптимальные параметры, веса), необходима функция ошибки (Loss Function) и алгоритм минимизации этой ошибки (Optimizer) - способ оптимизации весов.

https://pytorch.org/docs/stable/nn.html#loss-functions

https://pytorch.org/docs/stable/optim.html

**Обратное распространение ошибки** — это способ обучения нейронной сети. Цели обратного распространения просты: отрегулировать каждый вес пропорционально тому, насколько он способствует общей ошибке. Если мы будем итеративно уменьшать ошибку каждого веса, в конце концов у нас будет ряд весов, которые дают хорошие прогнозы.

Один цикл обучения:
1. Прогноз на тренировочной выборке 
2. Расчет ошибки на тренировочной выборке 
3. Корректировка параметрво модели (весов) с помощью обратного распространения ошибки.

<u>Для справки:</u><br>
1. **Обратное распространение ошибки** — это способ обучения нейронной сети. Цели обратного распространения просты: отрегулировать каждый вес пропорционально тому, насколько он способствует общей ошибке. Если мы будем итеративно уменьшать ошибку каждого веса, в конце концов у нас будет ряд весов, которые дают хорошие прогнозы.

2. `model.train()` устанавливает режим обучения (см. исходный код). Вы можете вызвать либо `model.eval()`, либо `model.train(mode=False)`, чтобы сообщить, что вы тестируете.

3. DataLoader - генератор батчей

4. `Torch.no_grad()` деактивирует движок autograd, что в итоге это уменьшит использование памяти и ускорит вычисления.

Используйте `Torch.no_grad()`, если:

<ul>
    <li>Для выполнения вывода без вычисления градиента.</li>
    <li>Быть уверенным, что никакие тестовые данные не попали в модель</li>
</ul>

In [12]:
for batch, (X, y) in enumerate(train_dataloader):
    print(X.size())
    print(X)
    print(type(X))
    print('-'*30)
    print(y.size())
    print(y)
    print(type(y))
    break

torch.Size([64, 1, 28, 28])
tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
   

In [13]:
[x.requires_grad for x in model.parameters()]

[True, True, True, True, True, True]

In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [15]:
# Метод обучения
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Прогноз
        pred = model(X)
        
        # Расчет ошибки
        loss = loss_fn(pred, y)
        
        # Обратное распространение ошибки
        optimizer.zero_grad() # занулили градиенты
        loss.backward() # вычисление градиента ошибки (grad U) относительно параметров модели
        optimizer.step() # обновляет параметры в модели после backward()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

Мы также проверяем эффективность модели на тестовом наборе данных, чтобы убедиться, что она обучается.

In [16]:
# Проверка на тестовом наборе данных
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset) # количество объектов в выборке
    numb_batches = len(dataloader) # количество батчей в выборке
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= numb_batches
    correct /= size
    print(f"Test Error: \nAccuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Процесс обучения проводится в течение нескольких итераций (эпох). В течение каждой эпохи модель изучает параметры, чтобы делать более точные прогнозы. Мы будем делать расчет точности и ошибки модели в каждую эпоху для того, чтобы отслеживать обучение. С увеличением эпохи мы хотим, чтобы точность увеличивалась, а величина ошибки уменьшалась.

In [17]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300014  [    0/60000]
loss: 2.293799  [ 6400/60000]
loss: 2.280757  [12800/60000]
loss: 2.278538  [19200/60000]
loss: 2.249225  [25600/60000]
loss: 2.228636  [32000/60000]
loss: 2.234422  [38400/60000]
loss: 2.207179  [44800/60000]
loss: 2.197369  [51200/60000]
loss: 2.168218  [57600/60000]
Test Error: 
Accuracy: 37.4%, Avg loss: 2.167024 

Epoch 2
-------------------------------
loss: 2.167990  [    0/60000]
loss: 2.165421  [ 6400/60000]
loss: 2.116159  [12800/60000]
loss: 2.137037  [19200/60000]
loss: 2.080992  [25600/60000]
loss: 2.024810  [32000/60000]
loss: 2.052163  [38400/60000]
loss: 1.984350  [44800/60000]
loss: 1.983029  [51200/60000]
loss: 1.916214  [57600/60000]
Test Error: 
Accuracy: 56.9%, Avg loss: 1.915761 

Epoch 3
-------------------------------
loss: 1.934323  [    0/60000]
loss: 1.916325  [ 6400/60000]
loss: 1.808177  [12800/60000]
loss: 1.853303  [19200/60000]
loss: 1.738515  [25600/60000]
loss: 1.689991  [32000/60000

Узнать больше об обучении моделей нейронных сетей в PyTorch можно в ноутбуке ***07_optimization_tutorial***.

--------------




Сохранение моделей
-------------
Распространенным способом сохранения модели является сериализация словаря внутреннего состояния (содержащего параметры модели).


In [18]:
torch.save(model.state_dict(), "./weights/weights_01.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


Loading Models
----------------------------

The process for loading a model includes re-creating the model structure and loading
the state dictionary into it.



In [19]:
model = NeuralNetwork()
model.load_state_dict(torch.load("./models/model_01.pth"))

<All keys matched successfully>

This model can now be used to make predictions.



In [20]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]

In [21]:
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"{pred[0].argmax(0)}")
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

9
Predicted: "Ankle boot", Actual: "Ankle boot"


Узнать больше об сохранении и загрузке моделей нейронных сетей ***08_saveloadrun_tutorial***.

---
